In [1]:
import torch
import random
import datetime
import pandas as pd
import numpy as np
import os

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, fit_treshold
from src.embeddings import RecsysEmbedding
from sklearn.linear_model import LogisticRegression

experiment_name = 'NeuralClickModel'
device = 'cuda:0'
seed = 7331
pkl_path = '../pkl/'


random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
def flatten(true, pred, mask, to_cpu=True):
    mask = mask.flatten()
    nnz_idx = mask.nonzero()[:, 0]
    true, pred = [x.flatten()[nnz_idx] for x in [true, pred]]
    if to_cpu:
        true, pred = [x.cpu().numpy() for x in [true, pred]]
    return true, pred

# Модель

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class NeuralClickModel(nn.Module):
    def __init__(self, embedding, readout=False):
        super().__init__()
        
        self.embedding = embedding
        
        self.emb_dim = embedding.embedding_dim
        self.rnn_layer = nn.GRU(
            input_size=self.emb_dim * 2, 
            hidden_size=self.emb_dim, 
            batch_first=True
        )
        self.out_layer = nn.Linear(self.emb_dim, 1)
        
        self.thr = -1.5
        self.readout = readout
        self.readout_mode = 'threshold' # ['soft' ,'threshold', 'sample', 'diff_sample']
        
        self.calibration = False
        self.w = 1
        self.b = 0

    def forward(self, batch):
        item_embs, user_embs = self.embedding(batch)
        shp = item_embs.shape
        max_sequence = item_embs.size(1)
        # ilya format:
        # 'items': (batch, slate, 2*embedding_dim ) 2, нужно для readout, по умолчанию ноли на половине эмбеддинга
        # 'clicks': (batch, slate)
        # 'users': (1, batch, embedding_dim), 
        # 'mask': (batch, slate)
        
        x = {}
        x['items'] = torch.cat(
            [
                item_embs.flatten(0,1),
                torch.zeros_like(item_embs.flatten(0,1)),
            ],
            dim = -1
        )
                
        if self.training:
            indices = (batch['length'] - 1)
        else:
            indices = (batch['in_length'] - 1)
        indices[indices<0] = 0
        indices = indices[:, None, None].repeat(1, 1, user_embs.size(-1))
        user_embs = user_embs.gather(1, indices).squeeze(-2).unsqueeze(0)
        x['users'] = user_embs.repeat_interleave(max_sequence, 1)
        x['clicks'] = (batch['responses'].flatten(0,1) > 0 ).int().clone()
        x['mask'] = batch['slates_mask'].flatten(0,1).clone()
        
        items = x['items']
        h = x['users']
        
        if self.readout:
            res = []
            seq_len = items.shape[1]
            for i in range(seq_len):
#                 print(items[:,[i],:])
                output, h = self.rnn_layer(items[:,[i],:], h)
                y = self.out_layer(output)[:, :, 0]
                
                # readout
                if i + 1 < seq_len:
                    if self.readout_mode == 'threshold':
                        items[:, [i+1], self.emb_dim:] *= (y.detach()[:, :, None] > self.thr).to(torch.float32)
                    elif self.readout_mode == 'soft':
                        items[:, [i+1], self.emb_dim:] *= torch.sigmoid(y)[:, :, None]
                    elif self.readout_mode == 'diff_sample' or self.readout_mode == 'sample':
                        eps = 1e-8
                        gumbel_sample = -( (torch.rand_like(y) + eps).log() / (torch.rand_like(y) + eps).log() + eps).log()
                        T = 0.3
                        bernoulli_sample = torch.sigmoid( (nn.LogSigmoid()(self.w * y + self.b) + gumbel_sample) / T )
                        if self.readout_mode == 'sample':
                            bernoulli_sample = bernoulli_sample.detach()
                        items[:, [i+1], self.emb_dim:] *= bernoulli_sample[:, :, None]
                    else:
                        raise
                    
                res.append(y)
        
            y = torch.cat(res, axis=1)
            
        else:
            items[:, 1:, self.emb_dim:] *= x['clicks'][:, :-1, None]
            rnn_out, _ = self.rnn_layer(items, h)
            y = self.out_layer(rnn_out)[:, :, 0]
        
        
        if self.calibration and self.training:
            clicks_flat, logits_flat = flatten(x['clicks'], y.detach(), x['mask'])
            logreg = LogisticRegression()
            logreg.fit(logits_flat[:, None], clicks_flat)
            γ = 0.3
            self.w = (1 - γ) * self.w + γ * logreg.coef_[0, 0]
            self.b = (1 - γ) * self.b + γ * logreg.intercept_[0]
            y = self.w * y + self.b
        else:
            y = self.w * y + self.b
            
        return y.reshape(shp[:-1])

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [4]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = NeuralClickModel(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='svd', embedding_dim = 2),
    readout=False
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device=device, lr=1e-3, num_epochs=5000, dummy=True,
    silent=True,
)


biulding affinity matrix...


3it [00:00, 3043.03it/s]


Test before learning: {'f1': 0.0, 'roc-auc': 0.3333333134651184, 'accuracy': 0.75}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.25 | f1: 0.4000000059604645 | auc: 0.3333333134651184 | treshold: 0.01
Test: accuracy: 0.25 | f1: 0.4000000059604645 | auc: 0.3333333134651184 | 
Val update: epoch: 41 |accuracy: 0.25 | f1: 0.4000000059604645 | auc: 0.6666666269302368 | treshold: 0.01
Test: accuracy: 0.25 | f1: 0.4000000059604645 | auc: 0.6666666269302368 | 
Val update: epoch: 44 |accuracy: 0.25 | f1: 0.4000000059604645 | auc: 1.0 | treshold: 0.01
Test: accuracy: 0.25 | f1: 0.4000000059604645 | auc: 1.0 | 
Val update: epoch: 84 |accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | treshold: 0.36000000000000004
Test: accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | 
Val update: epoch: 117 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.38
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


(NeuralClickModel(
   (embedding): RecsysEmbedding()
   (rnn_layer): GRU(4, 2, batch_first=True)
   (out_layer): Linear(in_features=2, out_features=1, bias=True)
 ),
 {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0})

# ContentWise

In [5]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=150)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

20216 data points among 108 batches


In [6]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    model = NeuralClickModel(
        RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings).to('cpu'),
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)


Evaluating NeuralClickModel with svd embeddings
Test before learning: {'f1': 0.17558717727661133, 'roc-auc': 0.4699956774711609, 'accuracy': 0.11125952005386353}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.42929893732070923 | f1: 0.20975828170776367 | auc: 0.6129906177520752 | treshold: 0.09
Test: accuracy: 0.4240484833717346 | f1: 0.2058972269296646 | auc: 0.6036361455917358 | 
Val update: epoch: 1 |accuracy: 0.6956035494804382 | f1: 0.22938480973243713 | auc: 0.6154359579086304 | treshold: 0.13
Test: accuracy: 0.6944913268089294 | f1: 0.22248786687850952 | auc: 0.6070038676261902 | 
Val update: epoch: 2 |accuracy: 0.7702124118804932 | f1: 0.23237289488315582 | auc: 0.6272104382514954 | treshold: 0.12
Test: accuracy: 0.7689383625984192 | f1: 0.2237672656774521 | auc: 0.616296648979187 | 
Val update: epoch: 3 |accuracy: 0.7970297932624817 | f1: 0.24255739152431488 | auc: 0.6442034244537354 | treshold: 0.12
Test: accuracy: 0.7981329560279846 | f1: 0.23128947615623474 | auc: 0.632957398891449 | 
Val update: epoch: 4 |accuracy: 0.7935561537742615 | f1: 0.24502098560333252 | auc: 0.6525763273239136 | treshold: 0.13
Test: accuracy: 0.7948450446128845 | f1: 0.

train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.204942524433136 | f1: 0.1885562390089035 | auc: 0.6216684579849243 | treshold: 0.09999999999999999
Test: accuracy: 0.20665208995342255 | f1: 0.18880383670330048 | auc: 0.6129509210586548 | 
Val update: epoch: 1 |accuracy: 0.7687110900878906 | f1: 0.25610679388046265 | auc: 0.661481499671936 | treshold: 0.15000000000000002
Test: accuracy: 0.7629056572914124 | f1: 0.2408234179019928 | auc: 0.6526389122009277 | 
Val update: epoch: 2 |accuracy: 0.7808392643928528 | f1: 0.2747199237346649 | auc: 0.6948072910308838 | treshold: 0.14
Test: accuracy: 0.7776130437850952 | f1: 0.26767846941947937 | auc: 0.6832988262176514 | 
Val update: epoch: 3 |accuracy: 0.8075683116912842 | f1: 0.29739895462989807 | auc: 0.7154268026351929 | treshold: 0.15000000000000002
Test: accuracy: 0.8043124079704285 | f1: 0.2838418483734131 | auc: 0.7015383243560791 | 
Val update: epoch: 4 |accuracy: 0.8339146971702576 | f1: 0.303112655878067 | auc: 0.7200019359588623 | treshold: 0.16
Te

In [7]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# RL4RS

In [8]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

45942 data points among 106 batches


In [9]:
for embeddings in ['explicit', 'neural', 'svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = NeuralClickModel(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings,
            embedding_dim=40
        ),
    ).to(device)

    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)


Evaluating NeuralClickModel with explicit embeddings
Test before learning: {'f1': 0.40662556886672974, 'roc-auc': 0.4045509099960327, 'accuracy': 0.3970983028411865}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.7775601148605347 | f1: 0.8413024544715881 | auc: 0.8329609632492065 | treshold: 0.41000000000000003
Test: accuracy: 0.7776568531990051 | f1: 0.8435452580451965 | auc: 0.8343753218650818 | 
Val update: epoch: 1 |accuracy: 0.7975378632545471 | f1: 0.8494938611984253 | auc: 0.8610242605209351 | treshold: 0.38
Test: accuracy: 0.8009430766105652 | f1: 0.8541408181190491 | auc: 0.8659748435020447 | 
Val update: epoch: 2 |accuracy: 0.7991583347320557 | f1: 0.8379138112068176 | auc: 0.875579833984375 | treshold: 0.42000000000000004
Test: accuracy: 0.8076411485671997 | f1: 0.847251296043396 | auc: 0.8831859230995178 | 
Val update: epoch: 3 |accuracy: 0.798747181892395 | f1: 0.8300206065177917 | auc: 0.8897788524627686 | treshold: 0.47000000000000003
Test: accuracy: 0.8094305396080017 | f1: 0.841908872127533 | auc: 0.8978234529495239 | 
Val update: epoch: 4 |accuracy: 0.822763979434967 | f1: 0.854157567024231 | auc: 0.8998100161552429 | treshold: 0.45
Test: accu

/opt/conda/envs/myenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


Val update: epoch: 28 |accuracy: 0.8433705568313599 | f1: 0.8735946416854858 | auc: 0.9146061539649963 | treshold: 0.34
Test: accuracy: 0.8491355180740356 | f1: 0.8796373009681702 | auc: 0.9195199012756348 | 
Val update: epoch: 29 |accuracy: 0.8451119661331177 | f1: 0.8778235912322998 | auc: 0.9146206378936768 | treshold: 0.36000000000000004
Test: accuracy: 0.8517228960990906 | f1: 0.884437084197998 | auc: 0.9188379049301147 | 

Evaluating NeuralClickModel with neural embeddings
Test before learning: {'f1': 0.5460179448127747, 'roc-auc': 0.45969700813293457, 'accuracy': 0.475661963224411}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.777003824710846 | f1: 0.8242805600166321 | auc: 0.8374440670013428 | treshold: 0.43
Test: accuracy: 0.7823963165283203 | f1: 0.8315014839172363 | auc: 0.8420712947845459 | 
Val update: epoch: 1 |accuracy: 0.7937164306640625 | f1: 0.8306763768196106 | auc: 0.8745828866958618 | treshold: 0.4
Test: accuracy: 0.7969048619270325 | f1: 0.8362799882888794 | auc: 0.8789252042770386 | 
Val update: epoch: 2 |accuracy: 0.8099936842918396 | f1: 0.8615341186523438 | auc: 0.8964874744415283 | treshold: 0.4
Test: accuracy: 0.8164671659469604 | f1: 0.8681879639625549 | auc: 0.9027137756347656 | 
Val update: epoch: 3 |accuracy: 0.8263435363769531 | f1: 0.8629457354545593 | auc: 0.8991527557373047 | treshold: 0.37
Test: accuracy: 0.8331277966499329 | f1: 0.8702892661094666 | auc: 0.9059569835662842 | 
Val update: epoch: 5 |accuracy: 0.8200551271438599 | f1: 0.8520050644874573 | auc: 0.8992465734481812 | treshold: 0.4
Test: accuracy: 0.8273001909255981 | f1: 0.8603495955

train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.7014946937561035 | f1: 0.7966386675834656 | auc: 0.7258546948432922 | treshold: 0.34
Test: accuracy: 0.7065650820732117 | f1: 0.8019648194313049 | auc: 0.722306489944458 | 
Val update: epoch: 1 |accuracy: 0.7291152477264404 | f1: 0.80112224817276 | auc: 0.7620196342468262 | treshold: 0.33
Test: accuracy: 0.732607901096344 | f1: 0.8059114813804626 | auc: 0.7602242827415466 | 
Val update: epoch: 2 |accuracy: 0.7455376386642456 | f1: 0.8166530132293701 | auc: 0.7819826006889343 | treshold: 0.36000000000000004
Test: accuracy: 0.7490509152412415 | f1: 0.821407675743103 | auc: 0.7805108428001404 | 
Val update: epoch: 3 |accuracy: 0.7546558380126953 | f1: 0.8142192363739014 | auc: 0.7934654951095581 | treshold: 0.36000000000000004
Test: accuracy: 0.7563051581382751 | f1: 0.8177247047424316 | auc: 0.7927939891815186 | 
Val update: epoch: 4 |accuracy: 0.7501088380813599 | f1: 0.827725350856781 | auc: 0.8100123405456543 | treshold: 0.38
Test: accuracy: 0.7549510

In [10]:
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

In [11]:
rl4rs_results

[{'f1': 0.884437084197998,
  'roc-auc': 0.9188379049301147,
  'accuracy': 0.8517228960990906,
  'embeddings': 'explicit'},
 {'f1': 0.8877638578414917,
  'roc-auc': 0.9230749607086182,
  'accuracy': 0.8541893362998962,
  'embeddings': 'neural'},
 {'f1': 0.8798442482948303,
  'roc-auc': 0.9140364527702332,
  'accuracy': 0.8463063836097717,
  'embeddings': 'svd'}]